In [5]:
%reload_ext autoreload
%autoreload 2
import collections, itertools, math
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg
import time, sys, os
from scipy import sparse

sys.path.append('..')
from pr3_utils import *

In [6]:
filename = "data/03.npz"
# filename = "./data/0027.npz"
# filename = "./data/0034.npz"
t, features, linear_velocity, rotational_velocity, K, b, cam_T_imu = load_data(filename)
# features = features[:, ::3, :]

In [9]:
from src.mapping import EKFLandmarkMapping

ekf = EKFLandmarkMapping(
    n_landmarks=features.shape[1],
    robot_cam_T_imu=cam_T_imu,
    robot_cam_intrinsic_calib=K,
    robot_cam_baseline=b,
    observation_noise_covariance= 100 * np.eye(4),
    prior_landmark_covariance= 5e-3 * np.eye(3),
    process_noise_covariance= 1e-3 * np.eye(6)
)


u = np.vstack([linear_velocity, rotational_velocity])
T = linear_velocity.shape[-1]
pose = []

for i in range(1, T):
    ekf.predict(u[:, i], (t[0, i] - t[0, i-1]))
    start = time.time()
    ekf.update(features[:, :, i])
    pose.append(np.linalg.inv(ekf.xU))
    end = time.time()

    #if i % 1 == 0:
    #    print(f'------------ {i} ----------------------')
    #    print('INITIALIZED LANDMARK:::', ekf.n_initialized)
    #    print('TIME COST::', round(end - start, 2))

pose = np.stack(pose, -1) if isinstance(pose, list) else pose
visualize_trajectory_2d(pose, ekf.xm, show_ori=True)

KeyboardInterrupt: 